In [23]:
from collections import OrderedDict

import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from torchvision.datasets import CIFAR10
import flwr as fl
from datetime import datetime
import importlib
import os
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pickle
import sys
import pandas as pd
sys.path.append(str(Path().absolute().parent.parent))

from src.models import nets
from src.data_loader import ALLDataset
torch.manual_seed(42)

In [27]:
with open('image_ids.pkl', 'rb') as handle:
    image_ids = pickle.load(handle)

In [35]:
len(image_ids['jarv']['val'])

1676

In [36]:
with open('image_ids_per_center/centralized_image_ids.pkl', 'rb') as handle:
    centralized_image_ids_dict = pickle.load(handle)
with open('image_ids_per_center/federated_image_ids.pkl', 'rb') as handle:
    federated_image_ids_dict = pickle.load(handle)

In [40]:
len(federated_image_ids_dict['jarv']['val'][1])

1676

In [41]:
len(federated_image_ids_dict['jarv']['train'][1])

6704

In [42]:
len(federated_image_ids_dict['jarv']['val'][1])+len(federated_image_ids_dict['jarv']['train'][1])

8380

In [ ]:
'1.2.826.0.1.3680043.9.3218.1.1.1405750.1758.1511969095974.2523.0' in federated_image_ids_dict['jarv']['val'][1]

In [ ]:
len(centralized_image_ids_dict['jarv']['val'][1])

In [ ]:
centralized_image_ids_dict['jarv']['val'][1][:5]

In [18]:
jarv_csv="/home/akis-linardos/Datasets/OPTIMAM/jarv_info.csv"
stge_csv="/home/akis-linardos/Datasets/OPTIMAM/stge_info.csv"
inbreast_csv="/home/lidia-garrucho/datasets/INBREAST/INbreast_updated_cropped_breast.csv"
cmmd_csv='/home/akis-linardos/Datasets/CMMD/info.csv'
bcdr_d01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D01_dataset/dataset_info.csv"
bcdr_d02_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-D02_dataset/dataset_info.csv"
bcdr_dn01_csv="/home/lidia-garrucho/datasets/BCDR/cropped/BCDR-DN01_dataset/dataset_info.csv"

j_csv=pd.read_csv(jarv_csv)
s_csv=pd.read_csv(stge_csv)
i_csv=pd.read_csv(inbreast_csv)
c_csv=pd.read_csv(cmmd_csv)
b_d01_csv=pd.read_csv(bcdr_d01_csv)
b_d02_csv=pd.read_csv(bcdr_d02_csv)
bcdr_csv = pd.concat([b_d01_csv, b_d02_csv])
center_to_csv = {'jarv': j_csv, 'stge': s_csv, 'inbreast': i_csv, 'cmmd': c_csv, 'bcdr':bcdr_csv}

In [ ]:
j_csv['study_id']=='1.2.826.0.1.3680043.9.3218.1.1.1447924.2040.1511973313408.4682.0'

In [ ]:
bcdr_csv

In [ ]:
j_csv.loc[j_csv['image_id'] == '1.2.826.0.1.3680043.9.3218.1.1.35791640.9441.1536882521446.102.0']['status'].item()

In [ ]:
type('jarv')==str

In [ ]:
len('BCDR-D01_dataset/')

In [ ]:
centralized_image_ids_dict[center]['val'][1][0][17:]

In [ ]:
center_to_csv['bcdr']['scan_path']

In [ ]:
center_to_csv['bcdr'].loc[center_to_csv['bcdr']['scan_path'] == 'patient_99/study_134/img_99_134_1_RCC.png']['classification'].iloc[0]

In [ ]:
def label_for_ids(dataset_pd, list_of_ids, label_header='BIRADS', center='bcdr'):
    benign, malign = 0, 0
    for img_id in list_of_ids:
        if center == 'bcdr':
            the_status = dataset_pd.loc[dataset_pd['scan_path'][17:] == img_id[17:]]['classification'].iloc[0]
        else:
            the_status = dataset_pd.loc[dataset_pd['image_id'] == img_id][label_header].iloc[0]
        if the_status == 'Benign' or (label=='BIRADS' and the_status in [2,3]):
            benign+=1
        elif the_status == 'Malignant' or the_status == 'Malign' or (label=='BIRADS' and the_status>3):
            malign+=1
    return {"Benign":benign, "Malign":malign}

In [ ]:
centers = ['jarv', 'stge', 'bcdr', 'cmmd', 'inbreast']

validation_sizes, validation_labels = {}, {}

for center in centers:
    val_l = centralized_image_ids_dict[center]['val'][1]
    validation_sizes[center]=len(val_l)
    if center == 'inbreast':
        label_header = 'BIRADS'
    else:
        label_header = 'status'
        
    validation_labels[center]=label_for_ids(center_to_csv[center], val_l, label_header=label_header, center=center)

In [ ]:
center

In [ ]:
validation_sizes

In [ ]:
validation_labels

In [ ]:
train_sizes, train_labels = {}, {}
for center in centers:
    train_l = centralized_image_ids_dict[center]['train'][1]
    train_sizes[center]=len(train_l)

In [ ]:
train_sizes